## Read data

In [ ]:
!pip install pylint flake8 pyright pandarallel

In [17]:
# Основные библиотеки
import os
import re
import io
import json
import tempfile
import subprocess
from pandarallel import pandarallel

# Работа с данными
import pandas as pd

# Многопоточность
from multiprocessing import Process, Queue
from queue import Empty

# Прогресс-бар
from tqdm import tqdm

# Контекстный менеджер для перенаправления вывода
from contextlib import redirect_stdout

pandarallel.initialize(progress_bar=True, nb_workers=8)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
path_to_data = "for_teams"
solutions_train = pd.read_excel(f"{path_to_data}/train/solutions.xlsx")
solutions_test = pd.read_excel(f"{path_to_data}/test/solutions.xlsx")
tests_train = pd.read_excel(f'{path_to_data}/train/tests.xlsx')
tasks_train = pd.read_excel(f'{path_to_data}/train/tasks.xlsx')
tests_test = pd.read_excel(f'{path_to_data}/test/tests.xlsx')
tasks_test = pd.read_excel(f'{path_to_data}/test/tasks.xlsx')

In [4]:
solutions_train.head(1)

,id,task_id,student_solution,author_comment,author_comment_embedding
0,13,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.2475823611021042 -1.3317935466766357 0.2535...


# Linter

```
├── solutions_train
│   ├── ...
│   ├── message_pylint // сообщение об ошибке pylint
│   ├── message_pyright // сообщение об ошибке pyright
|   ├── line_number_pyright // номер строки с ошибкой pyright
│   ├── line_pyright // строка с ошибкой pyright
│   ├── message_flake8 // сообщение об ошибке flake8
│   ├── flake_8_errors_cnt // количество ошибок flake8
│   ├── result_linter_ru // текст синтакс.ошибки для замены ответа чата (если заполнено)
```

In [24]:
solutions_train.iloc[1]['student_solution']

"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4', '#e4b3cd', '#e4e3b3', '#c0ced7']\ncite_project = ['#e4e3b3', '#a7a8f0', '#ccb1e6', '#b4f99e', '#f9b59e', '#c0ced7']\n\ncolor = input()\n\nif not(color in logo_project) and not(color in cite_project):\n    print(True)\nelse:\n    print(False)"

In [5]:
import ast
import pandas as pd

# Создаем новые столбцы, если они еще не существуют
if 'code_problem' not in solutions_train.columns:
    solutions_train['code_problem'] = False
if 'problem_message' not in solutions_train.columns:
    solutions_train['problem_message'] = ''
if 'line_with_code' not in solutions_train.columns:
    solutions_train['line_with_code'] = ''
if 'error_type' not in solutions_train.columns:
    solutions_train['error_type'] = ''

# Проходим по всему DataFrame и проверяем код в student_solution
for index, row in solutions_train.iterrows():
    try:
        ast.parse(row['student_solution'])
    except SyntaxError as e:
        solutions_train.at[index, 'code_problem'] = True
        solutions_train.at[index, 'problem_message'] = str(e)
        solutions_train.at[index, 'error_type'] = type(e).__name__

solutions_train.head(5)

,id,task_id,student_solution,author_comment,author_comment_embedding,code_problem,problem_message,line_with_code,error_type
0,13,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.2475823611021042 -1.3317935466766357 0.2535...,False,,,
1,14,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.25970256328582764 -1.4550446271896362 0.333...,False,,,
2,15,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.28293243050575256 -1.4774413108825684 0.243...,False,,,
3,16,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.44602400064468384 -0.845210611820221 -0.109...,False,,,
4,17,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Вы забыли поставить двоеточие после условия.,-0.2322002649307251 -1.3137400150299072 -0.383...,True,"expected ':' (<unknown>, line 8)",,SyntaxError


In [27]:
import ast
import pandas as pd

# Создаем новые столбцы, если они еще не существуют
if 'code_problem' not in solutions_test.columns:
    solutions_test['code_problem'] = False
if 'problem_message' not in solutions_test.columns:
    solutions_test['problem_message'] = ''
if 'line_with_code' not in solutions_test.columns:
    solutions_test['line_with_code'] = ''
if 'error_type' not in solutions_test.columns:
    solutions_test['error_type'] = ''

# Проходим по всему DataFrame и проверяем код в student_solution
for index, row in solutions_test.iterrows():
    try:
        ast.parse(row['student_solution'])
    except SyntaxError as e:
        solutions_test.at[index, 'code_problem'] = True
        solutions_test.at[index, 'problem_message'] = str(e)
        solutions_test.at[index, 'error_type'] = type(e).__name__

solutions_test.head(5)

,id,task_id,student_solution,author_comment,author_comment_embedding,message_pylint,message_pyright,line_number_pyright,line_pyright,message_flake8,flake_8_errors_cnt,result_linter_ru,error_open_tests,error_closed_tests,test_info,code_problem,problem_message,line_with_code,error_type
0,0,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,None,NaN,None,E265::block comment should start with '# ',1,,False,False,[],False,,,
1,1,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,E0001:Parsing failed: 'unterminated string lit...,"""("" was not closed",5.0,print(f'Реализация проекта будет стоить {money...,E999::SyntaxError: unterminated string literal...,1,"Синтаксическая ошибка. Значение ""«"" должно быт...",False,False,[],True,unterminated string literal (detected at line ...,,SyntaxError
2,2,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,E0001:Parsing failed: 'unterminated string lit...,"""("" was not closed",5.0,print(fРеализация проекта будет стоить {money}...,E999::SyntaxError: unterminated string literal...,1,"Синтаксическая ошибка. Значение ""«"" должно быт...",False,False,[],True,unterminated string literal (detected at line ...,,SyntaxError
3,3,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,None,NaN,None,E265::block comment should start with '# ',1,,False,False,[],False,,,
4,4,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,None,NaN,None,E265::block comment should start with '# ',1,,False,False,[],False,,,


Поиск синтаксических ошибок 3-мя линтерами

In [34]:
# Pyright
def run_pyright_on_string(code_string: str):
    try:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".py") as temp_file:
            temp_file_path = temp_file.name
            temp_file.write(code_string.encode("utf-8"))

        result = subprocess.run(
            ["pyright", "--outputjson", "--level", "error", "--project", ".", temp_file_path],
            capture_output=True,
            text=True,
        )

    except FileNotFoundError:
        print("Pyright is not installed or not found in your PATH.")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)

    return result.stdout


def get_basedpyright_errors(code_string: str):
    code_string = code_string.replace('input()', '""')
    basedpyright_output = run_pyright_on_string(code_string)
    lines = code_string.split("\n")
    errors = json.loads(basedpyright_output)["generalDiagnostics"]
    count = len(errors)
    if count == 0:
        return {"message": None, "line_number": None, "line": None}
    errors = errors[0]

    return {
        "message": errors["message"],
        "line_number": errors["range"]["start"]["line"],
        "line": lines[errors["range"]["start"]["line"]],
    }


#############################################################
# Pylint
def run_pylint_on_string(code_string: str):
    try:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".py") as temp_file:
            temp_file_path = temp_file.name
            temp_file.write(code_string.encode("utf-8"))

        result = subprocess.run(
            ["pylint", "--msg-template={msg_id}:{msg}", "--errors-only", temp_file_path],  # Используем pylint
            capture_output=True,
            text=True,
        )

    finally:
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)

    return result.stdout

def clean_pylint_output(output: str) -> str:
    # Убираем строки вида "************* Module tmppeqiq87k"
    output = re.sub(r"\*{13} Module \w+", "", output)

    # Убираем строки вида "(tmppeqiq87k, line 9)"
    output = re.sub(r"\(\w+, line \d+\)", "", output)

    # Убираем лишние пустые строки
    output = re.sub(r"\n\s*\n", "\n", output)

    return output

def get_pylint_errors(code_string: str):
    code_string = code_string.replace('input()', '""')
    pylint_output = run_pylint_on_string(code_string)

    pylint_output = clean_pylint_output(pylint_output).strip()
    return pylint_output


#############################################################
# Flake8
def run_flake8_on_string(code_string: str):
    try:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".py") as temp_file:
            temp_file_path = temp_file.name
            temp_file.write(code_string.encode("utf-8"))

        ignore_str = '--ignore=E265,F541,E202,E221,E231,E117,F841,W291,W292,W293,E501,E271,E275,E251,E211,E712,E111,E201,E222,E225,E227,E303'
        result = subprocess.run(
            ["flake8", '--format=%(code)s::%(text)s', ignore_str, temp_file_path],
            capture_output=True,
            text=True,
        )

    finally:
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)

    return result.stdout

def clean_flake8_output(output: str) -> str:

    # Убираем строки вида "line 9"
    output = re.sub(r"line \d+", "", output)

    # Убираем лишние пустые строки
    output = re.sub(r"\n\s*\n", "\n", output)

    return output

def get_flake8_errors(code_string: str):
    code_string = code_string.replace('input()', '""')
    flake8_output = run_flake8_on_string(code_string)

    errors = flake8_output.split('\n')
    errors = [x for x in errors if x.strip() != '']

    result = {"message": clean_flake8_output(errors[0]) if errors else '', "errors_cnt": len(errors)}
    return result

Хардкор синтаксических ошибок

In [37]:
def get_syntax_error(row):

    def contains_russian(text):
        return bool(re.search('[а-яА-Я]', text))

    errors_flake8 = {
        "F706::'return' outside function": "Ошибка в табуляции оператора return.",
        "E999::SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='?": "Синтаксическая ошибка. При сравнении двух значений используйте знак двойного равенства.",
        "E999::SyntaxError: cannot assign to subscript here. Maybe you meant '==' instead of '='?": "Синтаксическая ошибка. При сравнении двух значений следует использовать знако двойного равенства.",
        "F821::undefined name 'true'": "Обратите внимание на неверный синтаксис булевых значений.", # Синтаксическая ошибка?
        "F821::undefined name 'false'": "Обратите внимание на неверный синтаксис булевых значений.",
        "F821::undefined name '_'": "Вы забыли поставить знак нижнего подчеркивания в кавычки.",
        "E999::SyntaxError: unmatched ')'": "Вы забыли поставить открывающую скобку.",
        "E999::SyntaxError: '(' was never closed": "Вы забыли поставить закрывающую скобку."
    }
    errors_pyright = {
        '"__getitem__" method not defined on type "int"': "Ошибка при преобразовании переменной. int() не может применяться ко всему списку сразу. Примените int() к каждому элементу списка по отдельности.",
        "String literal is unterminated": "Синтаксическая ошибка. Значение должно быть заключено в кавычки одинакового формата.",
    }

    line = '' if pd.isna(row.line_pyright) else row.line_pyright
    line = line.strip().replace('\n', ' ')
    flake8_error = ('' if pd.isna(row.message_flake8) else row.message_flake8).strip()
    pylint_error = ('' if pd.isna(row.message_pylint) else row.message_pylint).strip()
    pyright_error = ('' if pd.isna(row.message_pyright) else row.message_pyright).strip()
    result = ''
    if flake8_error and flake8_error in errors_flake8:
        result = errors_flake8[flake8_error]

    elif flake8_error.startswith("E999::SyntaxError: expected ':'"):
        result = 'Вы забыли поставить двоеточие после '
        if line.startswith('while '):
            result += 'условия цикла while.'
        elif line.startswith('for '):
            result += 'условия цикла for.'
        elif line.startswith('def '):
            result += 'определения функции.'
        elif line.startswith('if '):
            result += 'условия if.'
        else:
            result += 'условия.'

    elif flake8_error.startswith("F821::undefined name "):
        undefined_name = flake8_error[21:].strip()
        if contains_russian(undefined_name):
            result = f'Вы забыли заключить "{undefined_name}" в кавычки.'

    elif flake8_error.startswith("E999::SyntaxError: invalid syntax") and pyright_error.startswith('Statements must be separated by newlines or semicolons'):
            result = 'Некорректный синтаксис при использовании функции open.'

    elif flake8_error.startswith("E999::SyntaxError: unterminated string literal"):
        quotes = [line.count('"'), line.count("'")]
        if sum(quotes) % 2 == 1:
            if 'f"' in line or "f'" in line:
                result = 'Вы забыли поставить закрывающую кавычку в f-строке.'
            else:
                result = 'Вы забыли поставить открывающую кавычку в f-строке.'
        else:
            result = 'Синтаксическая ошибка. Значение "«" должно быть заключено в кавычки одинакового формата.'

    elif flake8_error.startswith("E999::SyntaxError: invalid character"):
        result = 'Вы забыли поставить кавычку в f-строке.'

    elif flake8_error.startswith("E999::IndentationError: expected an indented block after"):
        substring = line.split()[0]
        result = f'Обратите внимание на некорректную табуляцию {substring}' #Некорректная табуляция внутри условной конструкции if.

    elif pyright_error and pyright_error in errors_pyright:
        result = errors_pyright[pyright_error]

    elif pyright_error.startswith('Cannot access attribute "add" for class "list[Unknown]" Attribute "add" is unknown'):
        result = "Некорректно использован метод добавления элементов в список. Список не имеет метода .add()"

    elif pyright_error.startswith("Operator ") and "not supported for types" in pyright_error:
        operator = pyright_error.split()[1].strip()[1:-1]
        if operator in ['<=', '<', '>', '>=']:
            result = f'Ошибка при использовании оператора {operator}. Сравнивать можно только переменные типа int. Скорректируйте условие if, для исправления ошибки.'

    return result

In [ ]:
solutions_train[["message_pylint"]] = solutions_train["student_solution"].parallel_apply(
    lambda x: pd.Series(get_pylint_errors(x))
)

solutions_train[["message_pyright", "line_number_pyright", "line_pyright"]] = solutions_train["student_solution"].parallel_apply(
    lambda x: pd.Series(get_basedpyright_errors(x))
)

solutions_train[["message_flake8", "flake_8_errors_cnt"]] = solutions_train["student_solution"].parallel_apply(
    lambda x: pd.Series(get_flake8_errors(x))
)

solutions_train["result_linter_ru"] = solutions_train.apply(get_syntax_error, axis=1)
solutions_train[~pd.isna(solutions_train['message_flake8'])].head(5)

In [30]:
solutions_test[["message_pylint"]] = solutions_test["student_solution"].parallel_apply(
    lambda x: pd.Series(get_pylint_errors(x))
)

solutions_test[["message_pyright", "line_number_pyright", "line_pyright"]] = solutions_test["student_solution"].parallel_apply(
    lambda x: pd.Series(get_basedpyright_errors(x))
)

solutions_test[["message_flake8", "flake_8_errors_cnt"]] = solutions_test["student_solution"].parallel_apply(
    lambda x: pd.Series(get_flake8_errors(x))
)

solutions_test["result_linter_ru"] = solutions_test.apply(get_syntax_error, axis=1)

solutions_test[~pd.isna(solutions_test['message_flake8'])].head(5)

,id,task_id,student_solution,author_comment,author_comment_embedding,message_pylint,message_pyright,line_number_pyright,line_pyright,message_flake8,flake_8_errors_cnt,result_linter_ru,error_open_tests,error_closed_tests,test_info,code_problem,problem_message,line_with_code,error_type
0,0,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,None,NaN,None,E265::block comment should start with '# ',1,,False,False,[],False,,,
1,1,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,E0001:Parsing failed: 'unterminated string lit...,"""("" was not closed",5.0,print(f'Реализация проекта будет стоить {money...,E999::SyntaxError: unterminated string literal...,1,"Синтаксическая ошибка. Значение ""«"" должно быт...",False,False,[],True,unterminated string literal (detected at line ...,,SyntaxError
2,2,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,E0001:Parsing failed: 'unterminated string lit...,"""("" was not closed",5.0,print(fРеализация проекта будет стоить {money}...,E999::SyntaxError: unterminated string literal...,1,"Синтаксическая ошибка. Значение ""«"" должно быт...",False,False,[],True,unterminated string literal (detected at line ...,,SyntaxError
3,3,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,None,NaN,None,E265::block comment should start with '# ',1,,False,False,[],False,,,
4,4,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,None,NaN,None,E265::block comment should start with '# ',1,,False,False,[],False,,,


## Tests

Пропускаем код студента через все тесты и добавляем указатель где код упал - в закрытых или открытых данных + тесты.
```
├── solutions_train
│   ├── ...
│   ├── error_open_test // упал в открытых True/False
│   ├── error_hidden_test //  упал в скрытых True/False
│   ├── test_info //  массив массивов, где во внутреннем массиве лежит информация - что на входе, что ожидалось, что пришло.
```
Проверяем только код, который компилируется

In [25]:
def replace_input_with_values(code, input_values):
    # Считаем количество вызовов input() в коде
    input_calls = len(re.findall(r'input\s*\(\s*\)', code))

    # Если нет вызовов input(), возвращаем исходный код
    if input_calls == 0:
        return code

    # Если input_values — строка, разделяем её на отдельные значения
    if isinstance(input_values, str):
        input_values_list = input_values.split(';')
    else:
        input_values_list = input_values

    # Заменяем каждый вызов input() на соответствующее значение
    if len(input_values_list) > 1:
        for value in input_values_list:
            code = re.sub(r'input\s*\(\s*\)', f'{value}', code, count=1)
    else:
        code = re.sub(r'input\s*\(\s*\)', f'"{input_values}"', code)
    return code

def replace_input_with_values(code, input_values):
    # Разделяем код на строки
    code_lines = code.split('\n')

    # Если input_values — строка, разделяем её на отдельные значения
    if isinstance(input_values, str):
        input_values_list = input_values.split('\n')
    else:
        input_values_list = input_values

    input_value_index = 0
    inside_loop = False
    new_code_lines = []

    is_loop_input = False
    for line in code_lines:
        # Проверяем, начался ли цикл
        if re.search(r'\bwhile\b|\bfor\b', line):
            inside_loop = True

        # Если находим вызов input()
        if 'input()' in line and inside_loop:
            is_loop_input = True

        # Проверяем конец цикла
        if inside_loop and re.search(r'\bbreak\b|\bcontinue\b|\bpass\b|\breturn\b', line):
            inside_loop = False

    inside_loop = False
    # if is_loop_input:
    #     new_code_lines.extend('')

    input_before_loop_cnt = 0
    for line in code_lines:
        # print(line, '|', inside_loop)
        # Проверяем, начался ли цикл
        if re.search(r'\bwhile\b|\bfor\b', line):
            inside_loop = True
            if is_loop_input:
                new_code_lines.append(f'input_index = -1')
                new_code_lines.append(f'input_values = {input_values_list[input_before_loop_cnt:]}')

        # Если находим вызов input()
        if 'input()' in line:
            if is_loop_input and inside_loop:
                new_code_lines.append(f'    input_index += 1')
                value = 'input_values[input_index]'
                is_loop_input = False
            else:
                if is_loop_input and not inside_loop:
                    input_before_loop_cnt += 1
                if input_value_index < len(input_values_list):
                    value = f'"{input_values_list[input_value_index]}"'
                    input_value_index += 1
                else:
                    value = '""'  # Если не хватает входных данных

            # Заменяем input() на текущее значение
            line = re.sub(r'input\s*\(\s*\)', f'{value}', line, count=1)

        new_code_lines.append(line)

        # Проверяем конец цикла
        if inside_loop and re.search(r'\bbreak\b|\bcontinue\b|\bpass\b|\breturn\b', line):
            inside_loop = False

    return '\n'.join(new_code_lines)

def execute_code_with_timeout(code, input_values, timeout=0.5):
    def target(queue, code, input_values):
        # Заменяем вызовы input() на передачу данных напрямую
        code = replace_input_with_values(code, input_values)
        # Перенаправляем стандартный вывод в буфер
        buffer = io.StringIO()
        with redirect_stdout(buffer):
            try:
                exec(code)
            except Exception as e:
                queue.put(f"Error: {str(e)}")
                return

        # Получаем результат вывода
        output = buffer.getvalue().strip()
        queue.put(output)

    q = Queue()
    p = Process(target=target, args=(q, code, input_values))
    p.start()
    p.join(timeout)

    if p.is_alive():
        p.terminate()
        p.join()
        return "Timeout"
    else:
        return q.get()

def checkout_tests(solutions, tests):
    # Добавляем колонки error_open и error_closed со значением False по умолчанию
    solutions['error_open_tests'] = False
    solutions['error_closed_tests'] = False
    solutions['test_info'] = [[] for _ in range(len(solutions))]  # Новая колонка для хранения информации о тестах

    # Проходим по каждой строке в solutions с использованием tqdm для отображения прогресса
    for index, row in tqdm(solutions.iterrows(), total=solutions.shape[0], desc="Processing solutions"):
        if not row['code_problem']:
            task_id = row['task_id']
            student_solution = row['student_solution']

            # Фильтруем tests по task_id
            relevant_tests = tests[tests['task_id'] == task_id]

            # Проходим по каждому тесту с использованием tqdm для отображения прогресса
            for test_index, test_row in relevant_tests.iterrows():

                test_id = test_row['id']
                test_input = test_row['input']
                expected_output = test_row['output']
                test_type = test_row['type']

                try:
                    actual_output = execute_code_with_timeout(student_solution, test_input)
                    # print(actual_output)
                    # Сравниваем результат с ожидаемым выводом
                    if "Timeout" not in actual_output and "Error" not in actual_output:
                        if actual_output != expected_output:
                            if test_type == 'open':
                                solutions.at[index, 'error_open_tests'] = True
                            elif test_type == 'closed':
                                solutions.at[index, 'error_closed_tests'] = True

                            # Добавляем информацию о тесте в колонку test_info
                            test_info = [test_input, expected_output, actual_output]
                            solutions.at[index, 'test_info'].append(test_info)
                except Exception as e:
                    pass

    return solutions

In [ ]:
solutions_train = checkout_tests(solutions_train, tests_train)
solutions_train.to_excel('data_with_tests_info_train18_10.xlsx')
solutions_train.head(3)

In [41]:
solutions_test = checkout_tests(solutions_test, tests_test)
solutions_test.to_excel('data_with_tests_info_test18_10.xlsx')
solutions_test.head(3)

Processing solutions: 100%|██████████| 325/325 [00:46<00:00,  7.02it/s]


,id,task_id,student_solution,author_comment,author_comment_embedding,message_pylint,message_pyright,line_number_pyright,line_pyright,message_flake8,flake_8_errors_cnt,result_linter_ru,error_open_tests,error_closed_tests,test_info,code_problem,problem_message,line_with_code,error_type
0,0,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,,None,NaN,None,,0,,False,False,[],False,,,
1,1,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,E0001:Parsing failed: 'unterminated string lit...,"""("" was not closed",5.0,print(f'Реализация проекта будет стоить {money...,E999::SyntaxError: unterminated string literal...,1,"Синтаксическая ошибка. Значение ""«"" должно быт...",False,False,[],True,unterminated string literal (detected at line ...,,SyntaxError
2,2,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,E0001:Parsing failed: 'unterminated string lit...,"""("" was not closed",5.0,print(fРеализация проекта будет стоить {money}...,E999::SyntaxError: unterminated string literal...,1,"Синтаксическая ошибка. Значение ""«"" должно быт...",False,False,[],True,unterminated string literal (detected at line ...,,SyntaxError


In [38]:
solutions_test.iloc[0]["test_info"]
#input/expected output/actual output

[['#a7f0ca', 'True', 'False'],
 ['#e4e3b3', 'False', 'True'],
 ['#a7a8f0', 'False', 'True'],
 ['#c0ced7', 'False', 'True'],
 ['#a7f0ca', 'True', 'False']]